In [13]:
import requests
import pandas as pd
import time

binance_url = "https://api.binance.com/api/v3/depth"

coins = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]
params = []

for i in range(len(coins)):
    param = {
        "symbol": coins[i],
        "limit": 100,
    }

    params.append(param)

btc_data = requests.get(binance_url, params=params[0]).json()
eth_data = requests.get(binance_url, params=params[1]).json()
sol_data = requests.get(binance_url, params=params[2]).json() 

In [14]:
btc_data

{'code': -1003,
 'msg': 'Way too much request weight used; IP banned until 1741272759135. Please use WebSocket Streams for live updates to avoid bans.'}

In [ ]:
binance_url = "https://api.binance.com/api/v3/depth"

coins = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]
params = [{"symbol": coin, "limit": 100} for coin in coins]

last_update_ids = {coin: None for coin in coins}

while True:
    coin = coins[0]
    response = requests.get(binance_url, params={"symbol": coin, "limit": 100})
    if response.status_code == 200:
        data = response.json()
        current_update_id = data.get('lastUpdateId')
        
        if last_update_ids[coin] is not None and current_update_id != last_update_ids[coin]:
            print(f"Change detected in {coin}: LastUpdateId changed from {last_update_ids[coin]} to {current_update_id}")
        
        last_update_ids[coin] = current_update_id
    else:
        print(f"Failed to fetch data for {coin}: {response.status_code}")
    
    time.sleep(1)  # Wait for 1 second before the next request

Change detected in BTCUSDT: LastUpdateId changed from 62985307445 to 62985309671
Change detected in BTCUSDT: LastUpdateId changed from 62985309671 to 62985310100
Change detected in BTCUSDT: LastUpdateId changed from 62985310100 to 62985310555
Change detected in BTCUSDT: LastUpdateId changed from 62985310555 to 62985310996
Change detected in BTCUSDT: LastUpdateId changed from 62985310996 to 62985313894


KeyboardInterrupt: 

In [3]:
import time
from datetime import datetime
time.time()
timestamp = 1742543252.98913 
print(datetime.fromtimestamp(timestamp))
timestamp = 1742543254.991134
print(datetime.fromtimestamp(timestamp))

2025-03-21 15:47:32.989130
2025-03-21 15:47:34.991134


In [ ]:
import asyncio
import websockets
import json

async def test_okx_subscription():
    try:
        async with websockets.connect("wss://ws.okx.com:8443/ws/v5/public") as ws:
            # 发送订阅请求
            subscribe_msg = {
                "op": "subscribe",
                "args": [{
                    "channel": "books",
                    "instId": "BTC-USDT"
                }]
            }
            await ws.send(json.dumps(subscribe_msg))

            # 接收订阅确认
            response = await ws.recv()
            print("订阅响应:", json.loads(response))

            # 持续接收数据
            while True:
                data = await ws.recv()
                print("收到数据:", json.loads(data))

    except Exception as e:
        print(f"发生错误: {e}")


async def main():
    # 创建并等待子任务
    task = asyncio.create_task(test_okx_subscription())
    await task
    print("Main task completed")

# 仅在程序入口调用 asyncio.run()
if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop